# 1. 환경설정

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
import time
import pickle
from tqdm import tqdm

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [14]:
import pickle
with open('certificated_stock_dic.pickle','rb') as f:
    certificated_stock_dic = pickle.load(f)

# 2. 일별 종가 데이터 수집 - 20개
키움API는 구름IDE에서 실행이 안되어 따로 웹크롤링

In [15]:
def get_stock_price(code, num_of_pages=2):
    url = f"http://finance.naver.com/item/sise_day.nhn?code={code}" 
    bs = BeautifulSoup(requests.get(url=url, headers = headers).text, 'html.parser')
    pgrr = bs.find("td", class_="pgRR")
    last_page = int(pgrr.a["href"].split('=')[-1])
    pages = min(last_page, num_of_pages) # 마지막 페이지와 가져올 페이지 수 중에 작은 값 선택
    df = pd.DataFrame()

    for page in range(1, pages+1):
        page_url = '{}&page={}'.format(url, page)
        df = df.append(pd.read_html(requests.get(page_url, headers={'User-agent': 'Mozilla/5.0'}).text)[0])
        
    df['일자'] = pd.to_datetime(df['날짜']) 
    df = df.dropna()
    df[['종가',  '시가', '고가', '저가', '거래량']] = df[['종가', '시가', '고가', '저가', '거래량']].astype(int) # int형으로 변경
    df = df[['일자', '거래량','시가', '고가', '저가','종가']] # 컬럼 순서 정렬
    df = df.sort_values(by = '일자') # 날짜순으로 정렬
    df = df.loc[::-1].reset_index(drop=True)
    return df

In [8]:
headers = {'User-agent': 'Mozilla/5.0'}
#pages = 2
df_sector_UTD = {}
for sector_name in sector:
    df_sector_UTD[sector_name] = {}


In [16]:
for sector_name, code_list in certificated_stock_dic.items():
    for stock_code in tqdm(code_list):
        df_sector_UTD[sector_name][stock_code] = get_stock_price(stock_code)
        time.sleep(0.01)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.22it/s]


In [27]:
df_sector_UTD['Food']['097950']

,일자,거래량,시가,고가,저가,종가
0,2023-08-18,30253,302000,306000,300000,303500
1,2023-08-17,38805,310000,310000,302500,307500
2,2023-08-16,36173,314000,318000,308500,309500
3,2023-08-14,54404,308500,316500,305000,314000
4,2023-08-11,17293,306000,311000,306000,308000
5,2023-08-10,46933,304500,311500,302500,310000
6,2023-08-09,42995,310000,310000,303000,305000
7,2023-08-08,153533,304000,317500,302000,310500
8,2023-08-07,48205,292000,301000,290000,297000
9,2023-08-04,23294,294500,296500,290000,294500


In [28]:
for sector_name, df_dict in df_sector_UTD.items():
    for stock_code, df in df_dict.items():
        df=df.drop_duplicates(subset=['일자'])
        df = df.sort_values(by=df.columns[0],ascending=True)
        df = df.reset_index(drop=True)
        df_sector_UTD[sector_name][stock_code] = df

In [14]:
'''for sector_name, df_dict in df_sector.items():
    for stock_code, df in df_dict.items():
        df = pd.concat([df_sector_UTD[sector_name][stock_code],df_sector[sector_name][stock_code]],axis=0)
        df=df.drop_duplicates(subset=['일자'])
        df = df.sort_values(by=df.columns[0],ascending=True)
        df = df.reset_index(drop=True)
        df_sector_UTD[sector_name][stock_code] = df'''

In [32]:
with open('df_sector_UTD.pickle','wb') as f:
    pickle.dump(df_sector_UTD,f)